In [1]:
#!pip3 install 'tensorflow==1.13.0rc1'

In [2]:
import tensorflow as tf
import numpy as np
from models import helpers

import pandas as pd
#import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [3]:
# Load Data
# Load Data
header = ['Context', 'Utterance', 'Label']
train_df = pd.read_csv("data/trainset.csv", names = header, nrows = 10000)
test_df = pd.read_csv("data/testset.csv", names = header, nrows = 3000)
validation_df = pd.read_csv("data/valset.csv", names = header, nrows = 3000)
y_test = np.zeros(len(test_df))

### Baseline Models

##### Baseline models for expected performance

In [4]:
def evaluate_recall(y, y_test, k=1):
    '''
    Evaluates recall@k (model picks k best responses out of 10). 
        If correct one is choosen test is marked as correct
    
    input:
        y: list of our predictions sorted by score in descending order
        y_test: label
        k: number of responses the model can pick (10 = recall of 100%)
    
    '''
    num_examples = float(len(y))
    num_correct = 0
    for predictions, label in zip(y, y_test):
        if label in predictions[:k]:
            num_correct += 1
    return num_correct/num_examples

In [5]:
# # Random Predictor - 
# def predict_random(context, utterances):
#     return np.random.choice(len(utterances), 10, replace=False)

In [6]:
# # Evaluate Random predictor 
# # to make sure the evaluate_recall function works properly

# y_random = [predict_random(test_df.Context[x], test_df.iloc[x,1:].values) for x in range(len(test_df))]
# y_test = np.zeros(len(y_random))
# for n in [1, 2, 5, 10]:
#     print('Recall @ ({}, 10): {:g}'.format(n, evaluate_recall(y_random, y_test, n)))

##### Baseline models for TF-IDF predictor and expected performance

In [7]:
class TFIDFPredictor:
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
 
    def train(self, data):
        self.vectorizer.fit(np.append(data.Context.values,data.Utterance.values))
 
    def predict(self, context, utterances):
        # Convert context and utterances into tfidf vector
        vector_context = self.vectorizer.transform({context})
        vector_doc = self.vectorizer.transform(utterances)
        # The dot product measures the similarity of the resulting vectors
        result = np.dot(vector_doc, vector_context.T).todense()
        result = np.asarray(result).flatten()
        # Sort by top results and return the indices in descending order
        return np.argsort(result, axis=0)[::-1]

In [8]:
train_df.Context[0]

"hey all, if i enable nvidia-glx on my system, the fonts in KDE get smaller ! ? strange, although the size hasn't changed </s> do you have a higher resolution or is it just small font </s>  it's the same resolution. The fonts in Opera don't change, it only happens in KDE </s> yes </s> i mean yes there is another way </s> you need to edit your fstab </s>  i know, but what can i write there ? </s> in the options section instead of defaults write somthing like this. rw,user     <- user makes it so anyone can mount it with full rw permissions"

In [9]:
# Evaluate TFIDF predictor ###FIXED
pred = TFIDFPredictor()
pred.train(train_df)
y = [pred.predict(test_df.Context[x], test_df.Utterance[x:]) for x in range(len(test_df))]
for n in [1, 2, 5, 10]:
    print('Recall @ ({}, 10): {:g}'.format(n, evaluate_recall(y, y_test, n)))

Recall @ (1, 10): 0.0123333
Recall @ (2, 10): 0.016
Recall @ (5, 10): 0.0313333
Recall @ (10, 10): 0.0443333


### Dual Encoder LSTM

**NOTE - seq2seq model would be another option instead of Dual Encoder**

In [10]:
pip install absl-py

Note: you may need to restart the kernel to use updated packages.


In [11]:
import tensorflow.app

In [12]:
FLAGS = tf.flags.FLAGS

In [13]:
def get_embeddings(hparams):
    '''
    We obtain vector representation for words.
    input: 
        hyperparameters
    output:
        Gets an existing variable with these parameters or create a new one
        
    '''
    if hparams.glove_path and hparams.vocab_path:
        tf.logging.info("Loading Glove embeddings...")
        vocab_array, vocab_dict = helpers.load_vocab(hparams.vocab_path)
        glove_vectors, glove_dict = helpers.load_glove_vectors(hparams.glove_path, vocab=set(vocab_array))
        initializer = helpers.build_initial_embedding_matrix(vocab_dict, glove_dict, glove_vectors, hparams.embedding_dim)
    else:
        tf.logging.info("No glove/vocab path specificed, starting with random embeddings.")
        initializer = tf.random_uniform_initializer(-0.25, 0.25)
    return tf.compat.v1.get_variable(
        "word_embeddings",
        shape=[hparams.vocab_size, hparams.embedding_dim],
        initializer=initializer)

In [14]:
def dual_encoder_model(
    hparams, 
    mode, 
    context,
    context_len,
    utterance,
    utterance_len,
    targets):
    
    # Initialize embedidngs randomly or with pre-trained vectors if available
    embeddings_W = get_embeddings(hparams)

    # Embed the context and the utterance
    context_embedded = tf.nn.embedding_lookup(
        embeddings_W, context, name="embed_context")
    utterance_embedded = tf.nn.embedding_lookup(
        embeddings_W, utterance, name="embed_utterance")

    # Build the RNN
    with tf.variable_scope("rnn") as vs:
        # We use an LSTM Cell
        cell = tf.nn.rnn_cell.LSTMCell(
            hparams.rnn_dim,
            forget_bias=2.0,
            use_peepholes=True,
            state_is_tuple=True)

        # Run the utterance and context through the RNN
        rnn_outputs, rnn_states = tf.nn.dynamic_rnn(
            cell,
            tf.concat(0, [context_embedded, utterance_embedded]),
            sequence_length=tf.concat(0, [context_len, utterance_len]),
            dtype=tf.float32)
        encoding_context, encoding_utterance = tf.split(0, 2, rnn_states.h)

    with tf.variable_scope("prediction") as vs:
        M = tf.get_variable("M", 
            shape=[hparams.rnn_dim, hparams.rnn_dim],
            initializer=tf.truncated_normal_initializer())

        # "Predict" a  response: c * M
        generated_response = tf.matmul(encoding_context, M)
        generated_response = tf.expand_dims(generated_response, 2)
        encoding_utterance = tf.expand_dims(encoding_utterance, 2)

        # Dot product between generated response and actual response
        # (c * M) * r
        logits = tf.batch_matmul(generated_response, encoding_utterance, True)
        logits = tf.squeeze(logits, [2])

        # Apply sigmoid to convert logits to probabilities
        probs = tf.sigmoid(logits)

        if mode == tf.contrib.learn.ModeKeys.INFER:
            return probs, None

        # Calculate the binary cross-entropy loss
        losses = tf.nn.sigmoid_cross_entropy_with_logits(logits, tf.to_float(targets))

    # Mean loss across the batch of examples
    mean_loss = tf.reduce_mean(losses, name="mean_loss")
    return probs, mean_loss

### Data Preprocessing

NOTE - There’s also tf.SequenceExample but it doesn’t seem to be supported by tf.learn yet (2016) - 


Esto sirve en caso de encontrar un DataSet con el que entrenar, preguntar mañana

In [15]:
import os
import csv
import itertools
import functools
import tensorflow as tf
import numpy as np
import array

In [16]:
FLAGS = tf.compat.v1.flags.FLAGS 

In [17]:
tf.compat.v1.flags.DEFINE_integer(
  "min_word_frequency", 5, "Minimum frequency of words in the vocabulary")

tf.compat.v1.flags.DEFINE_integer(
    "max_sentence_len", 160, "Maximum Sentence Length")

tf.compat.v1.flags.DEFINE_string(
  "input_dir", os.path.abspath("./data"),
  "Input directory containing original CSV data files (default = './data')")

tf.compat.v1.flags.DEFINE_string(
  "output_dir", os.path.abspath("./data"),
  "Output directory for TFrEcord files (default = './data')")

In [18]:
import sys #fix

In [19]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")]) #fix
assert(remaining_args == [sys.argv[0]]) #fix 

In [20]:
def tokenizer_fn(iterator):
    return (x.split(" ") for x in iterator)

In [21]:
def create_csv_iter(filename):
    """
    Returns an iterator over a CSV file. Skips the header.
    """
    with open(filename) as csvfile:
        reader = csv.reader(csvfile)
        # Skip the header
        next(reader)
        for row in reader:
            yield row

In [22]:
def create_vocab(input_iter, min_frequency):
    """
    Creates and returns a VocabularyProcessor object with the vocabulary
    for the input iterator.
    """
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
      FLAGS.max_sentence_len,
      min_frequency=min_frequency,
      tokenizer_fn=tokenizer_fn)
    return vocab_processor.fit(input_iter)

In [23]:
def transform_sentence(sequence, vocab_processor):
    """
    Maps a single sentence into the integer vocabulary. 
    Returns a python array.
    """
    return next(vocab_processor.transform([sequence])).tolist()

In [24]:
def create_text_sequence_feature(fl, sentence, sentence_len, vocab):
    """
    Writes a sentence to FeatureList protocol buffer
    """
    sentence_transformed = transform_sentence(sentence, vocab)
    for word_id in sentence_transformed:
        fl.feature.add().int64_list.value.extend([word_id])
    return fl

In [25]:
def create_example_train(row, vocab):
    """
    Creates a training example for the Ubuntu Dialog Corpus dataset.
    Returns a tensorflow Example Protocol Buffer object.
    """
    context, utterance, label = row
    context_transformed = transform_sentence(context, vocab)
    utterance_transformed = transform_sentence(utterance, vocab)
    context_len = len(next(vocab._tokenizer([context])))
    utterance_len = len(next(vocab._tokenizer([utterance])))
    label = int(float(label))

    # New Example
    example = tf.train.Example()
    example.features.feature["context"].int64_list.value.extend(context_transformed)
    example.features.feature["utterance"].int64_list.value.extend(utterance_transformed)
    example.features.feature["context_len"].int64_list.value.extend([context_len])
    example.features.feature["utterance_len"].int64_list.value.extend([utterance_len])
    example.features.feature["label"].int64_list.value.extend([label])
    return example

In [26]:
def create_example_test(row, vocab):
    """
    Creates a test/validation example for the Ubuntu Dialog Corpus dataset.  
    Returns a tensorflow Example Protocol Buffer object.
    """
    context, utterance = row[:2]
    distractors = row[2:]
    context_len = len(next(vocab._tokenizer([context])))
    utterance_len = len(next(vocab._tokenizer([utterance])))
    context_transformed = transform_sentence(context, vocab)
    utterance_transformed = transform_sentence(utterance, vocab)
    
    # New Example
    example = tf.train.Example()
    example.features.feature["context"].int64_list.value.extend(context_transformed)
    example.features.feature["utterance"].int64_list.value.extend(utterance_transformed)
    example.features.feature["context_len"].int64_list.value.extend([context_len])
    example.features.feature["utterance_len"].int64_list.value.extend([utterance_len])

    # Distractor sequences
    for i, distractor in enumerate(distractors):
        dis_key = "distractor_{}".format(i)
        dis_len_key = "distractor_{}_len".format(i)
        # Distractor Length Feature
        dis_len = len(next(vocab._tokenizer([distractor])))
        example.features.feature[dis_len_key].int64_list.value.extend([dis_len])
        # Distractor Text Feature
        dis_transformed = transform_sentence(distractor, vocab)
        example.features.feature[dis_key].int64_list.value.extend(dis_transformed)

    return example


In [27]:
def create_tfrecords_file(input_filename, output_filename, example_fn):
    """
    Creates a TFRecords file for the given input data and
    example transofmration function
    """
    writer = tf.python_io.TFRecordWriter(output_filename)
    print("Creating TFRecords file at {}...".format(output_filename))
    for i, row in enumerate(create_csv_iter(input_filename)):
        x = example_fn(row)
        writer.write(x.SerializeToString())
    writer.close()
    print("Wrote to {}".format(output_filename))

In [28]:
def write_vocabulary(vocab_processor, outfile):
    """
    Writes the vocabulary to a file, one word per line.
    """
    vocab_size = len(vocab_processor.vocabulary_)
    with open(outfile, "w") as vocabfile:
        for id in range(vocab_size):
            word =  vocab_processor.vocabulary_._reverse_mapping[id]
            vocabfile.write(word + "\n")
    print("Saved vocabulary to {}".format(outfile))
    
    if __name__ == "__main__":
        print("Creating vocabulary...")
        input_iter = create_csv_iter("data/trainset.csv")
        input_iter = (x[0] + " " + x[1] for x in input_iter)
        vocab = create_vocab(input_iter, min_frequency=FLAGS.min_word_frequency)
        print("Total vocabulary size: {}".format(len(vocab.vocabulary_)))

    # Create vocabulary.txt file
    write_vocabulary(
        vocab, os.path.join(FLAGS.output_dir, "vocabulary.txt"))

    # Save vocab processor
    vocab.save(os.path.join(FLAGS.output_dir, "vocab_processor.bin"))

    # Create validation.tfrecords
    create_tfrecords_file(
          input_filename=VALIDATION_PATH,
          output_filename=os.path.join(FLAGS.output_dir, "validation.tfrecords"),
          example_fn=functools.partial(create_example_test, vocab=vocab))

    # Create test.tfrecords
    create_tfrecords_file(
          input_filename=TEST_PATH,
          output_filename=os.path.join(FLAGS.output_dir, "test.tfrecords"),
          example_fn=functools.partial(create_example_test, vocab=vocab))

    # Create train.tfrecords
    create_tfrecords_file(
          input_filename=TRAIN_PATH,
          output_filename=os.path.join(FLAGS.output_dir, "train.tfrecords"),
          example_fn=functools.partial(create_example_train, vocab=vocab))

###  Input function

In [29]:
import tensorflow as tf

In [30]:
TEXT_FEATURE_SIZE = 160

In [31]:
def get_feature_columns(mode):
    feature_columns = []

    feature_columns.append(tf.contrib.layers.real_valued_column(
        column_name="context", dimension=TEXT_FEATURE_SIZE, dtype=tf.int64))
    feature_columns.append(tf.contrib.layers.real_valued_column(
        column_name="context_len", dimension=1, dtype=tf.int64))
    feature_columns.append(tf.contrib.layers.real_valued_column(
        column_name="utterance", dimension=TEXT_FEATURE_SIZE, dtype=tf.int64))
    feature_columns.append(tf.contrib.layers.real_valued_column(
        column_name="utterance_len", dimension=1, dtype=tf.int64))

    if mode == tf.contrib.learn.ModeKeys.TRAIN:
        # During training we have a label feature
        feature_columns.append(tf.contrib.layers.real_valued_column(
            column_name="label", dimension=1, dtype=tf.int64))

    if mode == tf.contrib.learn.ModeKeys.EVAL:
        # During evaluation we have distractors
        for i in range(9):
            feature_columns.append(tf.contrib.layers.real_valued_column(
                column_name="distractor_{}".format(i), dimension=TEXT_FEATURE_SIZE, dtype=tf.int64))
            feature_columns.append(tf.contrib.layers.real_valued_column(
                column_name="distractor_{}_len".format(i), dimension=1, dtype=tf.int64))

    return set(feature_columns)


In [32]:
def create_input_fn(mode, input_files, batch_size, num_epochs):
    
    def input_fn():
        
        '''
        Create a feature definition that describes the fields in our Example file
        Read records from the input_files with tf.TFRecordReader
        Parse the records according to the feature definition
        Extract the training labels
        Batch multiple examples and training labels
        Return the batched examples and training labels
        '''
        
        features = tf.contrib.layers.create_feature_spec_for_parsing(
            get_feature_columns(mode))

        feature_map = tf.contrib.learn.io.read_batch_features(
            file_pattern=input_files,
            batch_size=batch_size,
            features=features,
            reader=tf.TFRecordReader,
            randomize_input=True,
            num_epochs=num_epochs,
            queue_capacity=200000 + batch_size * 10,
            name="read_batch_features_{}".format(mode))

        # This is an ugly hack because of a current bug in tf.learn
        # During evaluation TF tries to restore the epoch variable which isn't defined during training
        # So we define the variable manually here
        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            tf.get_variable(
            "read_batch_features_eval/file_name_queue/limit_epochs/epochs",
            initializer=tf.constant(0, dtype=tf.int64))

        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            target = feature_map.pop("label")
        else:
          # In evaluation we have 10 classes (utterances).
          # The first one (index 0) is always the correct one
            target = tf.zeros([batch_size, 1], dtype=tf.int64)
        return feature_map, target #batched_features, labels 
    return input_fn

### Defining Evaluation Metrics

In [33]:
#import tensorflow as tf
#import functools
from tensorflow.contrib.learn.python.learn.metric_spec import MetricSpec #(DEPRECATED)
#from tensorflow.estimator.EstimatorSpec.eval_metric_ops import MetricSpec ## substitute would be fix

In [34]:
def create_evaluation_metrics():
    '''
    Functools.partial is to convert a function that takes 3 arguments 
    to one that only takes 2 arguments. 
    streaming_sparse_recall_at_k:
        Streaming: metric is accumulated over multiple batches
        Sparse: format of our labels.
    '''
    eval_metrics = {}
    for k in [1, 2, 5, 10]:
        eval_metrics["recall_at_%d" % k] = MetricSpec(metric_fn=functools.partial(
            tf.contrib.metrics.streaming_sparse_recall_at_k,
            k=k))
    return eval_metrics

In [35]:
create_evaluation_metrics()

Instructions for updating:
Use tf.estimator.EstimatorSpec.eval_metric_ops.


{'recall_at_1': <tensorflow.contrib.learn.python.learn.metric_spec.MetricSpec at 0x133dae650>,
 'recall_at_2': <tensorflow.contrib.learn.python.learn.metric_spec.MetricSpec at 0x133dae890>,
 'recall_at_5': <tensorflow.contrib.learn.python.learn.metric_spec.MetricSpec at 0x133daec90>,
 'recall_at_10': <tensorflow.contrib.learn.python.learn.metric_spec.MetricSpec at 0x133daee10>}

In [36]:
#Metric Spec se tiene que cambiar por Estimator? y ya?

### hparams

Custom object that holds hyperparameters, nobs we can tweak, of our model. This hparams object is given to the model when we instantiate it.

In [37]:
#import tensorflow as tf
from collections import namedtuple

In [38]:
# Model Parameters
tf.flags.DEFINE_integer(
  "vocab_size",
  91620,
  "The size of the vocabulary. Only change this if you changed the preprocessing")

# Model Parameters
tf.flags.DEFINE_integer("embedding_dim", 100, "Dimensionality of the embeddings")
tf.flags.DEFINE_integer("rnn_dim", 256, "Dimensionality of the RNN cell")
tf.flags.DEFINE_integer("max_context_len", 160, "Truncate contexts to this length")
tf.flags.DEFINE_integer("max_utterance_len", 80, "Truncate utterance to this length")

# Pre-trained embeddings
tf.flags.DEFINE_string("glove_path", None, "Path to pre-trained Glove vectors")
tf.flags.DEFINE_string("vocab_path", None, "Path to vocabulary.txt file")

# Training Parameters
tf.compat.v1.flags.DEFINE_float("learning_rate", 0.001, "Learning rate")
tf.compat.v1.flags.DEFINE_integer("batch_size", 128, "Batch size during training")
tf.compat.v1.flags.DEFINE_integer("eval_batch_size", 16, "Batch size during evaluation")
tf.compat.v1.flags.DEFINE_string("optimizer", "Adam", "Optimizer Name (Adam, Adagrad, etc)")

FLAGS = tf.compat.v1.flags.FLAGS

In [39]:
HParams = namedtuple(
  "HParams",
  [
    "batch_size",
    "embedding_dim",
    "eval_batch_size",
    "learning_rate",
    "max_context_len",
    "max_utterance_len",
    "optimizer",
    "rnn_dim",
    "vocab_size",
    "glove_path",
    "vocab_path"
  ])

In [41]:
def create_hparams():
    return HParams(
        batch_size=FLAGS.batch_size,
        eval_batch_size=FLAGS.eval_batch_size,
        vocab_size=FLAGS.vocab_size,
        optimizer=FLAGS.optimizer,
        learning_rate=FLAGS.learning_rate,
        embedding_dim=FLAGS.embedding_dim,
        max_context_len=FLAGS.max_context_len,
        max_utterance_len=FLAGS.max_utterance_len,
        glove_path=FLAGS.glove_path,
        vocab_path=FLAGS.vocab_path,
        rnn_dim=FLAGS.rnn_dim)

### Boilerplate Code

In [42]:
import os
import time
import itertools
import tensorflow as tf
# import udc_model
# import udc_hparams
# import udc_metrics
# import udc_inputs
#from models.dual_encoder import dual_encoder_model


**FLAGS**: a way to give command line parameters to the program (similar to Python’s argparse)

In [43]:
tf.compat.v1.flags.DEFINE_string("input_dir", "./data", "Directory containing input data files 'train.tfrecords' and 'validation.tfrecords'")
tf.compat.v1.flags.DEFINE_string("model_dir", None, "Directory to store model checkpoints (defaults to ./runs)")
tf.compat.v1.flags.DEFINE_integer("loglevel", 20, "Tensorflow log level")
tf.compat.v1.flags.DEFINE_integer("num_epochs", None, "Number of training Epochs. Defaults to indefinite.")
tf.compat.v1.flags.DEFINE_integer("eval_every", 2000, "Evaluate after this many train steps")

FLAGS = tf.compat.v1.flags.FLAGS

TIMESTAMP = int(time.time())

if FLAGS.model_dir:
    MODEL_DIR = FLAGS.model_dir
else:
    MODEL_DIR = os.path.abspath(os.path.join("./runs", str(TIMESTAMP)))

TRAIN_FILE = os.path.abspath(os.path.join(FLAGS.input_dir, "train.tfrecords"))
VALIDATION_FILE = os.path.abspath(os.path.join(FLAGS.input_dir, "validation.tfrecords"))

tf.logging.set_verbosity(FLAGS.loglevel)


DuplicateFlagError: The flag 'input_dir' is defined twice. First from /usr/local/lib/python3.7/site-packages/ipykernel_launcher.py, Second from /usr/local/lib/python3.7/site-packages/ipykernel_launcher.py.  Description from first occurrence: Input directory containing original CSV data files (default = './data')

In [44]:
def main(unused_argv):
    hparams = create_hparams()

    model_fn = create_model_fn(
        hparams,
        model_impl=dual_encoder_model)

    estimator = tf.contrib.learn.Estimator(
        model_fn=model_fn,
        model_dir=MODEL_DIR,
        config=tf.contrib.learn.RunConfig())

    input_fn_train = create_input_fn(
        mode=tf.contrib.learn.ModeKeys.TRAIN,
        input_files=[TRAIN_FILE],
        batch_size=hparams.batch_size,
        num_epochs=FLAGS.num_epochs)

    input_fn_eval = create_input_fn(
        mode=tf.contrib.learn.ModeKeys.EVAL,
        input_files=[VALIDATION_FILE],
        batch_size=hparams.eval_batch_size,
        num_epochs=1)

    eval_metrics = create_evaluation_metrics()

    eval_monitor = tf.contrib.learn.monitors.ValidationMonitor(
            input_fn=input_fn_eval,
            every_n_steps=FLAGS.eval_every,
            metrics=eval_metrics)

    estimator.fit(input_fn=input_fn_train, steps=None, monitors=[eval_monitor])

    if __name__ == "__main__":
        tf.app.run()

### Creating the model
udc_model.py

In [45]:
import tensorflow as tf
import sys

In [46]:
def get_id_feature(features, key, len_key, max_len):
    ids = features[key]
    ids_len = tf.squeeze(features[len_key], [1])
    ids_len = tf.minimum(ids_len, tf.constant(max_len, dtype=tf.int64))
    return ids, ids_len

In [47]:
def create_train_op(loss, hparams):
    return tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=hparams.learning_rate,
        clip_gradients=10.0,
        optimizer=hparams.optimizer)

In [48]:
def create_model_fn(hparams, model_impl):

    def model_fn(features, targets, mode):
        context, context_len = get_id_feature(
            features, "context", "context_len", hparams.max_context_len)

        utterance, utterance_len = get_id_feature(
            features, "utterance", "utterance_len", hparams.max_utterance_len)

        batch_size = targets.get_shape().as_list()[0]

        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            probs, loss = model_impl(
              hparams,
              mode,
              context,
              context_len,
              utterance,
              utterance_len,
              targets)
            train_op = create_train_op(loss, hparams)
            return probs, loss, train_op

        if mode == tf.contrib.learn.ModeKeys.INFER:
            probs, loss = model_impl(
              hparams,
              mode,
              context,
              context_len,
              utterance,
              utterance_len,
              None)
            return probs, 0.0, None

        if mode == tf.contrib.learn.ModeKeys.EVAL:

            # We have 10 exampels per record, so we accumulate them
            all_contexts = [context]
            all_context_lens = [context_len]
            all_utterances = [utterance]
            all_utterance_lens = [utterance_len]
            all_targets = [tf.ones([batch_size, 1], dtype=tf.int64)]

        for i in range(9):
            distractor, distractor_len = get_id_feature(features,
                "distractor_{}".format(i),
                "distractor_{}_len".format(i),
                hparams.max_utterance_len)
            all_contexts.append(context)
            all_context_lens.append(context_len)
            all_utterances.append(distractor)
            all_utterance_lens.append(distractor_len)
            all_targets.append(
              tf.zeros([batch_size, 1], dtype=tf.int64)
            )

        probs, loss = model_impl(
              hparams,
              mode,
              tf.concat(0, all_contexts),
              tf.concat(0, all_context_lens),
              tf.concat(0, all_utterances),
              tf.concat(0, all_utterance_lens),
              tf.concat(0, all_targets))

        split_probs = tf.split(0, 10, probs)
        shaped_probs = tf.concat(1, split_probs)

        # Add summaries
        tf.histogram_summary("eval_correct_probs_hist", split_probs[0])
        tf.scalar_summary("eval_correct_probs_average", tf.reduce_mean(split_probs[0]))
        tf.histogram_summary("eval_incorrect_probs_hist", split_probs[1])
        tf.scalar_summary("eval_incorrect_probs_average", tf.reduce_mean(split_probs[1]))

        return shaped_probs, loss, None

    return model_fn

In [50]:
model_fn = create_model_fn(hparams=create_hparams(), model_impl=dual_encoder_model)

In [52]:
model_fn()

TypeError: model_fn() missing 3 required positional arguments: 'features', 'targets', and 'mode'